In [ ]:
import os
import pandas as pd
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist

# Create folders if they don't exist
os.makedirs("data/raw", exist_ok=True)
os.makedirs("output", exist_ok=True)

# Read the Excel files for students and employers (now repo-relative)
student_data = pd.read_excel("data/raw/AY_2021_BBE_ES&P_Free_Response.xlsx")
employer_data = pd.read_excel("data/raw/AY_2021_BBE_EFR_Free_Response.xlsx")

# Merge the datasets based on 'Responder ID' (inner join)
merged_data = student_data.merge(
    employer_data[['Responder ID', "What are the student's strengths?"]],
    on='Responder ID',
    how='inner'
)

# Preprocess function for text
def preprocess_text(text):
    if isinstance(text, str):
        text = text.translate(str.maketrans('', '', string.punctuation))
        tokens = word_tokenize(text.lower())
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        return ' '.join(tokens)
    else:
        return ''

# Keyword extraction function
def get_top_keywords(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    freq_dist = FreqDist(tokens)
    return [word for word, freq in freq_dist.most_common(10)]

# Preprocess the merged comments
merged_data["goal_comments"] = merged_data["Please reflect on how one aspect of this co-op experience relates to a personal, academic, or professional goal that you are pursuing at Drexel. Be specific about both your goal and how one aspect of the co-op relates to this goal. Suggested length: 250-300 words."].apply(preprocess_text)

merged_data["strengths_comments"] = merged_data["What are the student's strengths?"].apply(preprocess_text)

# Extract top keywords
merged_data['goal_top_keywords'] = merged_data['goal_comments'].apply(get_top_keywords)
merged_data['strengths_top_keywords'] = merged_data['strengths_comments'].apply(get_top_keywords)

# Save the DataFrame to a new Excel file
merged_data.to_excel("output/merged_data_with_top_keywords.xlsx", index=False)